In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [47]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  2.0  minutos y 27.594858646392822  segundos


In [48]:
#Agrupamiento por relación de trabajo
inicio = time.time()
agrupa = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
               (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
               (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                     '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
               (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                    '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
               (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
               (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
tipo = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select( agrupa, tipo, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 24.04534077644348  segundos


In [5]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       'INDEPENDIENTES',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PRIVADO',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PUBLICO',
       '108-ARTISTA Y GESTOR DE CULTURA', '06-CODIGO DEL TRABAJO - CT '],
      dtype=object)

In [6]:
afi['SECTOR_A'].unique()

array(['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR'], dtype=object)

In [7]:
afi.shape

(68062270, 13)

In [77]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()

In [78]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

114212

In [10]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 41.88606286048889  segundos
Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 45.65396595001221  segundos
Tiempo de ejecución es:  0.0  horas con  3.0  minutos y 27.485018968582153  segundos


(64756154, 14)

In [11]:
del afi
del ced_counts
afi_fil

,CEDULA_COD,ANIO,MES,CODSEC,SECTOR,DESTIPEMP,DESRELTRA,CODCII,PROVINCIA,CANTON,SALARIO,PORCENTAJE_AP,SECTOR_A,FECHA
26481585,126,2004,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,147.45,20.5,PRI,2004-12-01
67442511,126,2005,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,41.0,PRI,2005-01-01
41179435,126,2005,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-02-01
44033860,126,2005,3,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-03-01
47099991,126,2005,4,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896256,23594465,2017,11,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14045.72,20.6,PRI,2017-11-01
5107034,23594465,2017,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14164.25,20.6,PRI,2017-12-01
44333802,23594465,2018,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,12903.21,20.6,PRI,2018-01-01
38102314,23594465,2018,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,42432.21,20.6,PRI,2018-02-01


In [12]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [13]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Contar los meses, considerando duplicidad en los meses que puede tener aporte simultaneos
caract['MES_C'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )
#Meses trabajados unicos
aux = afi_sel.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])
caract['MES_T'] = caract['CEDULA_COD'].map( aux['CEDULA_COD'].value_counts() )

In [14]:
caract

,CEDULA_COD,INI_HL,FIN_HL,MES_C,MES_T
0,126,2004-12-01,2010-12-01,73,73
1,152,2004-07-01,2009-09-01,63,63
2,216,2004-07-01,2009-09-01,98,63
3,230,2004-12-01,2015-05-01,126,126
4,235,2004-08-01,2011-06-01,83,83
...,...,...,...,...,...
444112,23075971,2017-02-01,2022-10-01,66,66
444113,23402574,2010-01-01,2019-02-01,110,110
444114,23456125,2016-07-01,2022-03-01,69,69
444115,23503686,2004-09-01,2011-11-01,87,87


In [15]:
#Personas que tienen varios aportes simultaneos
caract[caract['MES_C']>caract['MES_T']]

,CEDULA_COD,INI_HL,FIN_HL,MES_C,MES_T
2,216,2004-07-01,2009-09-01,98,63
10,397,2004-07-01,2012-01-01,138,91
12,419,2004-11-01,2009-12-01,115,58
20,461,2004-07-01,2010-09-01,221,75
23,472,2004-09-01,2012-01-01,176,89
...,...,...,...,...,...
444102,20949683,2011-08-01,2019-07-01,92,91
444104,21035240,2010-03-01,2019-09-01,112,111
444105,21035907,2006-07-01,2013-08-01,68,67
444106,21104059,2014-07-01,2023-02-01,105,104


In [16]:
#afi_sel[afi_sel['CEDULA_COD']==21729781].tail(60)

In [17]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [18]:
aux = afi_sel[afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]
aux1 = afi_sel[~afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]

In [19]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tienen más de un sector
inicio = time.time()
afi_sel_g = aux.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                            'NUM_SEC_MES': lambda x: len(x),
                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                }).reset_index()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


Tiempo de ejecución es:  0.0  horas con  10.0  minutos y 27.963502645492554  segundos


In [80]:
afi_sel_g

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR


In [32]:
afi_sel_g_all = pd.concat( [aux1[['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SECTOR_A', 'NUM_SEC_MES', '%_NUM_SECTOR']], 
                            afi_sel_g], ignore_index=True )

In [39]:
afi_sel_g_all = afi_sel_g_all.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel_g_all.loc[ (afi_sel_g_all['NUM_SEC_MES'] == 1), '%_NUM_SECTOR'] = '1'

In [82]:
afi_sel_g_all

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR
0,126,2010,12,8.06,PRI,1,1
1,126,2010,11,250.00,PRI,1,1
2,126,2010,10,250.00,PRI,1,1
3,126,2010,9,250.00,PRI,1,1
4,126,2010,8,250.00,PRI,1,1
...,...,...,...,...,...,...,...
59831331,23594465,2012,9,10528.81,PRI,1,1
59831332,23594465,2012,8,10722.25,PRI,1,1
59831333,23594465,2012,7,10722.25,PRI,1,1
59831334,23594465,2012,6,13046.07,PRI,1,1


In [ ]:
del aux
del aux1

In [87]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel_g_all['GRUPO'] = (afi_sel_g_all.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel_g_all['SAL_PROM_GRUPO'] = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')


In [139]:
afi_sel_g_all

SyntaxError: invalid syntax (3539575301.py, line 1)

In [128]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==216]['GRUPO'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [137]:
#Se obtienen los sueldos promedios para cada grupo y cada cedula
inicio = time.time()
df1 = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].unique().reset_index().copy()
df1['SAL_PROM_GRUPO'] = df1['SAL_PROM_GRUPO'].apply(lambda x: x[0])
df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  3.0  minutos y 3.0261378288269043  segundos


In [140]:
df1.head(60)

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
1,126,2,250.000000
2,126,3,241.666667
0,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000
5,126,6,161.670000
6,126,7,147.450000
7,152,1,689.693333
8,152,2,618.975000
9,152,3,491.187500


In [96]:
inicio = time.time()
top = 5
top_sal = df1.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
indices = top_sal.index
df2 = df1.loc[indices].reset_index(drop=True).copy()


Tiempo de ejecución es:  0.0  horas con  7.0  minutos y 48.987428426742554  segundos


1            250.000000
2            241.666667
0            229.838333
3            170.000000
4            161.670000
               ...     
5388748    17158.475000
5388746    17023.186667
5388745    16884.076667
5388744    16205.699167
5388747    15750.057500
Name: SAL_PROM_GRUPO, Length: 2220161, dtype: float64

In [112]:
df1.head(60)

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
1,126,2,250.000000
2,126,3,241.666667
0,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000
5,126,6,161.670000
6,126,7,147.450000
7,152,1,689.693333
8,152,2,618.975000
9,152,3,491.187500


In [114]:
df2.head(60)

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
0,126,2,250.000000
1,126,3,241.666667
2,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000
5,152,1,689.693333
6,152,2,618.975000
7,152,3,491.187500
8,152,4,407.137500
9,152,5,337.130000


In [116]:
#Función para seleccionar los grupos de sueldos
combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
afi_sel_g_all['SEL_GRUPO'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(afi_sel_g_all['CEDULA_COD'], afi_sel_g_all['GRUPO'])]
afi_sel_g_all['SEL_GRUPO'] = afi_sel_g_all['SEL_GRUPO'].astype(int)

In [125]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==216].tail(60)

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,SEL_GRUPO
139,216,2009,6,641.34,PUB,1,1,1,625.315000,1
140,216,2009,5,641.34,PUB,1,1,1,625.315000,1
141,216,2009,4,641.34,PUB,1,1,1,625.315000,1
142,216,2009,3,641.34,PUB,1,1,1,625.315000,1
143,216,2009,2,641.34,PUB,1,1,1,625.315000,1
144,216,2009,1,641.34,PUB,1,1,1,625.315000,1
145,216,2008,12,577.24,PUB,1,1,1,625.315000,1
146,216,2008,11,577.24,PUB,1,1,1,625.315000,1
147,216,2008,10,577.24,PUB,1,1,1,625.315000,1
148,216,2008,9,577.24,PUB,1,1,2,550.830833,1
